# Demo on active testing from paper
Active Testing: Sample–Efficient Model Evaluation

Active Surrogate Estimators: An Active Learning Approach to Label–Efficient Model Evaluation

In [1]:
import os, sys
import torch, json
import numpy as np

from main import build_model_main
from util.slconfig import SLConfig
from datasets import build_dataset
from util.visualizer import COCOVisualizer
from util import box_ops
import pickle
import copy

In [41]:
result_path = "./results/deep_ensemble/"
consider_model_results = {"DINO_0011_4scale.pkl", "DINO_0011_5scale.pkl", "DINO_0023_4scale.pkl", "DINO_0022_5scale.pkl", "DINO_0033_4scale.pkl", "DINO_0031_5scale.pkl"}
ego_model_index = 5
model_nums = len(consider_model_results)

## Load model results

In [3]:
# ['pred_boxes', 'pred_logits', 'scores', 'labels', 'boxes']
deep_ensemble_results = []
for path in consider_model_results:
    temp_path = result_path + path
    with open(temp_path, "rb") as outfile:
         deep_ensemble_results.append(pickle.load(outfile))

## Load data set

In [4]:
scale_config = "5scale"
model_config_path = "config/DINO/DINO_" + scale_config + ".py" # change the path of the model config file
args = SLConfig.fromfile(model_config_path) 
args.device = 'cuda' 
args.dataset_file = 'coco'
args.coco_path = "../coco/" # the path of coco
args.fix_size = False
dataset_val = build_dataset(image_set='val', args=args)

data_aug_params: {
  "scales": [
    480,
    512,
    544,
    576,
    608,
    640,
    672,
    704,
    736,
    768,
    800
  ],
  "max_size": 1333,
  "scales2_resize": [
    400,
    500,
    600
  ],
  "scales2_crop": [
    384,
    600
  ]
}
loading annotations into memory...
Done (t=0.49s)
creating index...
index created!


## Model results postprocess
Include: remove prediction with low score, prediction matching

In [17]:
# output_dict = ['pred_boxes', 'pred_logits', 'scores', 'labels', 'boxes']
score_threshold = 0
max_num_select = 300
pro_results = copy.deepcopy(deep_ensemble_results)
for agent in range(model_nums):
    for img_idx in range(len(pro_results[agent])):
        pred_logits = pro_results[agent][img_idx]['pred_logits']
        pred_boxes = pro_results[agent][img_idx]['pred_boxes']
        if pred_logits is None:
            continue
        # only consider prediction with score larger than score_threshold
        select_mask = pred_logits > score_threshold
        select_idx_all = np.reshape(select_mask, -1).nonzero()[0]
        select_idx = select_idx_all // pred_logits.shape[2]
        assert len(select_idx) <= max_num_select
        if len(select_idx) <= 0:
            pro_results[agent][img_idx]['pred_logits']  = None
            pro_results[agent][img_idx]['lables']  = None
            pro_results[agent][img_idx]['scores']  = None
            pro_results[agent][img_idx]['pred_boxes']  = None
            continue
        lables = select_idx_all % pred_logits.shape[2]
        scores = pred_logits[select_mask]
        pred_boxes = pred_boxes[:, select_idx]
        pred_logits = pred_logits[:, select_idx]
        pro_results[agent][img_idx]['pred_logits'] = torch.from_numpy(pred_logits)
        pro_results[agent][img_idx]['lables'] = torch.from_numpy(lables)
        pro_results[agent][img_idx]['scores'] = torch.from_numpy(scores)
        pro_results[agent][img_idx]['pred_boxes'] = torch.from_numpy(pred_boxes)

In [18]:
pro_results[0][0]['pred_boxes'].shape, pro_results[0][0]['pred_logits'].shape, pro_results[0][0]['lables'].shape, pro_results[0][0]['scores'].shape

(torch.Size([1, 9, 4]),
 torch.Size([1, 9, 91]),
 torch.Size([9]),
 torch.Size([9]))

In [72]:
from util.box_ops import box_cxcywh_to_xyxy, generalized_box_iou
from scipy.optimize import linear_sum_assignment
def hungarian_matching(outputs, targets, cost_class = 2.0, cost_bbox = 5.0, cost_giou = 2.0, focal_alpha = 0.25):
    """ Performs the matching
    Params:
        outputs/targets: This is a dict that contains at least these entries:
             "pred_logits": Tensor of dim [batch_size, num_queries, num_classes] with the classification logits
             "pred_boxes": Tensor of dim [batch_size, num_queries, 4] with the predicted box coordinates
             "lables": Tensor of dim [num_queries] with the label of each predicted box
    Returns:
        A list of size batch_size, containing tuples of (index_i, index_j) where:
            - index_i is the indices of the selected predictions (in order)
            - index_j is the indices of the corresponding selected targets (in order with high priority)
        For each batch element, it holds:
            len(index_i) = len(index_j) = min(num_queries, num_target_boxes)
    """
    if outputs is None or targets is None or outputs["pred_logits"] is None or targets["pred_logits"] is None:
        return []
    assert outputs["pred_logits"].shape[0] == targets["pred_logits"].shape[0]
    bs, num_queries = outputs["pred_logits"].shape[:2]
    
    # We flatten to compute the cost matrices in a batch
    out_prob = outputs["pred_logits"].flatten(0, 1).sigmoid()  # [batch_size * num_queries, num_classes]
    out_bbox = outputs["pred_boxes"].flatten(0, 1)  # [batch_size * num_queries, 4]
    
    tgt_ids = targets["lables"]
    tgt_bbox = targets["pred_boxes"].flatten(0, 1)
    
    # Compute the classification cost.
    alpha = focal_alpha
    gamma = 2.0
    neg_cost_class = (1 - alpha) * (out_prob ** gamma) * (-(1 - out_prob + 1e-8).log())
    pos_cost_class = alpha * ((1 - out_prob) ** gamma) * (-(out_prob + 1e-8).log())
    cost_class = pos_cost_class[:, tgt_ids] - neg_cost_class[:, tgt_ids]
    
    # Compute the L1 cost between boxes
    cost_bbox = torch.cdist(out_bbox, tgt_bbox, p=1)
    
    # Compute the giou cost betwen boxes            
    cost_giou = -generalized_box_iou(box_cxcywh_to_xyxy(out_bbox), box_cxcywh_to_xyxy(tgt_bbox))
    
    # Final cost matrix
    C = cost_bbox * cost_bbox + cost_class * cost_class + cost_giou * cost_giou
    C = C.view(bs, num_queries, -1)
    
    num_target_boxes = targets["pred_logits"].shape[1]
    indices = [linear_sum_assignment(c[i]) for i, c in enumerate(C.split(num_target_boxes, -1))]
    indices = [indices[0][0][np.argsort(indices[0][1])], np.sort(indices[0][1])]
    return [torch.as_tensor(indices[0], dtype=torch.int64), torch.as_tensor(indices[1], dtype=torch.int64)]

In [73]:
indices = hungarian_matching(pro_results[0][0], pro_results[3][0])
indices

[tensor([2, 6, 0, 4, 1, 3, 5, 8, 7]), tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])]

In [74]:
for img_idx in range(len(pro_results[agent])):
    for agent in range(model_nums):
        if agent == ego_model_index:
            continue
        match_indices = hungarian_matching(pro_results[agent][img_idx], pro_results[ego_model_index][img_idx])

## Acquisition Function

In [ ]:
# ICML Active testing

In [8]:
deep_ensemble_results[0][0]['pred_logits'].shape

(1, 900, 91)

In [28]:
deep_ensemble_results[0][0]['pred_boxes'].shape

(1, 900, 4)

In [10]:
deep_ensemble_results[0][0]['scores'].shape

(300,)

In [11]:
deep_ensemble_results[0][0]['labels'].shape

(300,)

In [12]:
deep_ensemble_results[0][0]['boxes'].shape

(300, 4)

In [30]:
dataset_val[0][1].keys()

dict_keys(['boxes', 'labels', 'image_id', 'area', 'iscrowd', 'orig_size', 'size'])

In [32]:
dataset_val[0][1]['boxes']

tensor([[0.3896, 0.4161, 0.0386, 0.1631],
        [0.1276, 0.5052, 0.2333, 0.2227],
        [0.9342, 0.5835, 0.1271, 0.1848],
        [0.6047, 0.6325, 0.0875, 0.2414],
        [0.5025, 0.6273, 0.0966, 0.2312],
        [0.6692, 0.6190, 0.0471, 0.1910],
        [0.5128, 0.5283, 0.0337, 0.0272],
        [0.6864, 0.5320, 0.0829, 0.3240],
        [0.6125, 0.4462, 0.0236, 0.0839],
        [0.8119, 0.5017, 0.0230, 0.0375],
        [0.7863, 0.5364, 0.0317, 0.2542],
        [0.9562, 0.7717, 0.0224, 0.1073],
        [0.9682, 0.7781, 0.0201, 0.1090],
        [0.7106, 0.3100, 0.0218, 0.0514],
        [0.8866, 0.8316, 0.0573, 0.2105],
        [0.5569, 0.5167, 0.0178, 0.0529],
        [0.6517, 0.5288, 0.0150, 0.0294],
        [0.3880, 0.4784, 0.0222, 0.0414],
        [0.5338, 0.4879, 0.0152, 0.0393],
        [0.6000, 0.6471, 0.1962, 0.2088]])

In [24]:
outputs = deep_ensemble_results[0][0]
target_sizes = torch.Tensor([[1.0, 1.0]])
out_logits, out_bbox = torch.Tensor(outputs['pred_logits']), torch.Tensor(outputs['pred_boxes'])

In [20]:
num_select = 300
nms_iou_threshold = -1
not_to_xyxy = False
test = True
assert len(out_logits) == len(target_sizes)

prob = out_logits.sigmoid()
topk_values, topk_indexes = torch.topk(prob.view(out_logits.shape[0], -1), num_select, dim=1)
scores = topk_values
topk_boxes = topk_indexes // out_logits.shape[2]
labels = topk_indexes % out_logits.shape[2]
if not_to_xyxy:
    boxes = out_bbox
else:
    boxes = box_ops.box_cxcywh_to_xyxy(out_bbox)

if test:
    assert not not_to_xyxy
    boxes[:,:,2:] = boxes[:,:,2:] - boxes[:,:,:2]
boxes = torch.gather(boxes, 1, topk_boxes.unsqueeze(-1).repeat(1,1,4))

# and from relative [0, 1] to absolute [0, height] coordinates
img_h, img_w = target_sizes.unbind(1)
scale_fct = torch.stack([img_w, img_h, img_w, img_h], dim=1)
boxes = boxes * scale_fct[:, None, :]

if nms_iou_threshold > 0:
    item_indices = [nms(b, s, iou_threshold=nms_iou_threshold) for b,s in zip(boxes, scores)]

    results = [{'scores': s[i], 'labels': l[i], 'boxes': b[i]} for s, l, b, i in zip(scores, labels, boxes, item_indices)]
else:
    results = [{'scores': s, 'labels': l, 'boxes': b} for s, l, b in zip(scores, labels, boxes)]
print(results)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  if sys.path[0] == "":


AttributeError: 'list' object has no attribute 'unbind'

In [25]:
prob.view(out_logits.shape[0], -1).shape

torch.Size([1, 81900])

In [26]:
outputs['pred_logits'].shape

(1, 900, 91)

In [27]:
900*91

81900